In [ ]:
import difflib


class Library:
    def __init__(self, file_name='books.txt'):
        self.file_name = file_name
        self.file = open(file_name, 'a+')

    def __del__(self):
        self.file.close()

    def list_books(self):
        self.file.seek(0)
        books = self.file.read().splitlines()
        if not books:
            print("Kütüphanede hiç kitap yok.")
            return
        for book in books:
            book_details = book.split(',')
            print(f"Kitap Adı: {book_details[0]}, Yazar: {book_details[1]}, Yayın Yılı: {book_details[2]}, Sayfa Sayısı: {book_details[3]}")

    def add_book(self):
        title = input("Kitap başlığı: ")
        author = input("Yazar: ")
        release_year = input("İlk yayın yılı: ")
        pages = input("Sayfa sayısı: ")
        book_line = f"{title},{author},{release_year},{pages}\n"
        self.file.write(book_line)
        self.file.flush()

    def remove_book(self):
        self.file.seek(0)
        books = self.file.read().splitlines()
        title_to_remove = input("Kaldırılacak kitap başlığı: ").strip()
        book_titles = [book.split(',')[0] for book in books]

        # 1. Adım: Tam eşleşme var mı kontrol ediyoruz
        exact_matches = [book for book in books if book.split(',')[0].lower() == title_to_remove.lower()]
        # 1.1 - Tam eşleşme sağlayan tek bir kitap varsa
        if len(exact_matches) == 1:
            books.remove(exact_matches[0])
            self._rewrite_books(books)
            print(f"'{exact_matches[0].split(',')[0]}' isimli kitap listeden çıkarıldı.")
            return
        # 1.2 - Tam eşleşme sağlayan birden fazla kitap varsa
        elif len(exact_matches) > 1:
          print("Aynı isimde birden fazla kitap bulundu. Hangisini silmek istersiniz?")
          for index, match in enumerate(exact_matches):
            parts = match.split(',')
            book_title = parts[0]
            author_name = parts[1]
            print(f"{index + 1}) {book_title} - {author_name}")
          self._select_and_remove(books, exact_matches)
          return

        # 2. Adım: Tam eşleşme yoksa
        # 2.1 İlk olarak iki string arasındaki benzerlik oranını bulmak için (2.2. Adım'da kullanacağımız) bir fonksiyon tanımlıyoruz
        def custom_similarity_score(str1, str2):
            str1, str2 = str1.lower(), str2.lower()
            common_chars = set(str1) & set(str2)
            common_chars_count = sum(min(str1.count(char), str2.count(char)) for char in common_chars)
            length_difference = abs(len(str1) - len(str2))
            common_ratio = common_chars_count / float(max(len(str1), len(str2)))
            similarity_score = common_ratio - (length_difference / max(len(str1), len(str2)))
            return max(0, similarity_score)

        # 2.2 Ardından, benzer eşleşmeler var mı kontrol ediyoruz
        close_matches = difflib.get_close_matches(title_to_remove, book_titles, n=5, cutoff=0.6)
        # Benzer en az bir eşleşme varsa
        if close_matches:
            print("Bu isimde bir kitap bulunamadı. Benzer isimli kitaplar ise şunlar:")

            # Her bir eşleşmenin benzerlik oranını (2.1. Adım'da tanımladığımız fonksiyonu kullanarak) hesaplıyoruz
            match_ratios = [(match, custom_similarity_score(title_to_remove, match)) for match in close_matches]

            # Benzerlik oranlarına göre bunları sıralıyoruz
            sorted_matches = sorted(match_ratios, key=lambda x: x[1], reverse=True)

            # Benzerlik oranlarına göre sıralanmış kitapların bütün bilgilerini içeren bir liste oluşturuyoruz
            books_to_select_from = []
            for match, _ in sorted_matches:
                matching_books = [book for book in books if book.split(',')[0] == match]
                books_to_select_from.extend(matching_books)

            # Sıralanmış kitapların isimlerini, yazarlarının isimlerini ve benzerlik oranlarını yazdırıyoruz
            for index, book in enumerate(books_to_select_from):
                book_details = book.split(',')
                print(f"{index + 1}) {book_details[0]} - {book_details[1]} (Benzerlik oranı: {sorted_matches[index][1]:.2f})")

            # Sıralanmış kitaplardan hangisini listeden kaldırmak istediğini kullanıcıya sormak üzere kitap kaldırma fonksiyonunu çalıştırıyoruz
            self._select_and_remove(books, books_to_select_from)
            return
        # Benzer hiç eşleşme yoksa
        else:
            print(f"Listede '{title_to_remove}' isimli bir kitap bulunamadı.")


    def _select_and_remove(self, books, matches):
        choice = input("Silmek istediğiniz kitabın numarasını girin, iptal için '0' tuşlayın: ")
        if choice.isdigit():
            choice = int(choice)
            if choice == 0:
                print("İşlem iptal edildi.")
            elif 0 < choice <= len(matches):
                book_to_remove = matches[choice - 1]
                books.remove(book_to_remove)
                self._rewrite_books(books)
                print(f"'{book_to_remove.split(',')[0]}' başlıklı kitap listeden çıkarıldı.")
            else:
                print("Geçersiz bir seçim yaptınız.")
        else:
            print("Lütfen geçerli bir numara giriniz.")

    def _rewrite_books(self, books):
        self.file.close()
        with open(self.file_name, 'w') as self.file:
            for book in books:
                self.file.write(f"{book}\n")
        self.file = open(self.file_name, 'a+')

def main():
    lib = Library()
    while True:
        print("*** MENÜ ***\n1) Kitapları Listele\n2) Kitap Ekle\n3) Kitap Kaldır\nq) Çıkış Yap")
        choice = input("Seçiminiz: ")
        if choice == '1':
            lib.list_books()
        elif choice == '2':
            lib.add_book()
        elif choice == '3':
            lib.remove_book()
        elif choice.lower() == 'q':
            print("Programdan çıkılıyor...")
            break
        else:
            print("Geçersiz seçim.")

if __name__ == "__main__":
    main()